In [ ]:
import gym
import minihack
import time
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as display
from tqdm import tqdm
import random
import math

# set the seed for reproducibility
SEED = 42
np.random.seed(SEED)

from utils import *
from gen import test, random_search, true_random_search, true_random_nsteps


In [ ]:
env = gym.make(
    "MiniHack-Navigation-Custom-v0",
    observation_keys=("chars", "pixel"),
    des_file = "complex_maze.des",
    max_episode_steps=10000,
)
state = env.reset()
env.render()

In [ ]:
plt.imshow(state['pixel'])

In [ ]:
a = (0, -1)
b = (0, 1)
c = (1, 0)      
d = (-1, 0)

ACTIONS = [a, b, c, d]


env = gym.make(
    "MiniHack-Navigation-Custom-v0",
    observation_keys=("chars", "pixel"),
    des_file = "complex_maze.des",
    max_episode_steps=10000,
)

state = env.reset()
game_map = state['chars']
game = state['pixel']
start = get_player_location(game_map)
target = get_target_location(game_map)


def modify_action(t1, t2):
    """Sum two tuples"""
    return (t1[0] + t2[0], t1[1] + t2[1])

def is_crossoverable(action1, action2):
    # if action 1 and action 2 are oblique, return False
    dx = abs(action1[0] - action2[0])
    dy = abs(action1[1] - action2[1])
    # return false if the two components are changing together and > 1
    if (dx > 0 and dy > 0) and (dx > 1 or dy > 1):
        return False
    else:
        #print(f'actions: ,{action1, action2}')
        return True

def crossover_path(path1, path2):
    """Crossover two paths"""
    # randomly select a crossover point
    i = np.random.randint(1, min(len(path1), len(path2)))
    while not is_crossoverable(path1[i-1], path2[i]):
        i = np.random.randint(1, min(len(path1), len(path2)))

    # return the two paths joined at the crossover point
    # TODO:  implement controls on move validity

    # until the path is valid, merge the 2 path
    """print(f'point of crossover: {i}')
    print(f'path1: {path1}')
    print(f'path2: {path2}')
    print(f'lenp1: {len(path1[:i])}, path1[:i]: {path1[:i]}')
    print(f'lenp2: {len(path2[i:])}, path2[i:]: {path2[i:]}')"""

    pathtry = path1[:i] + path2[i:]

    #concatenete path1 and path2

    for idx in range(1, len(pathtry)):
        if is_wall(game_map[pathtry[idx]]):
            #truncate here pathtry[:idx]
            return path1[:i] + path2[i:idx-1]
    return pathtry

def crossover(actions1, actions2):
    """Crossover two paths"""
    # randomly select a crossover point
    i = np.random.randint(1, min(len(actions1), len(actions2)))
    # return the two paths joined at the crossover point
    return actions1[:i] + actions2[i:]

def mutate_path(path, mutation_rate=0.05):
    """Mutate a path"""
    # randomly select n postions to mutate
    print(f'before mutation {path}')
    actions = actions_from_path(start, path[1:])
    idxs = random.sample(list(range(len(actions)))[1:], k = math.floor(len(actions)/10))
    print("idxs", idxs)
    # randomly select new actions for each position and replace
    # TODO?  implement controls on move validity
    for idx in idxs:
        print(f"valid moves:", get_valid_actions(game_map, path[idx]))
        print(game_map[path[idx]])
        action = random.choice(get_valid_actions(game_map, path[idx]))
        actions[idx] = action
    path = path_from_actions(path[0], actions)
    print(f'after mutation {path}')
    return path

def mutate(actions, mutation_rate=0.05):
    """Mutate a path"""
    # randomly select n postions to mutate
    idxs = random.sample(list(range(len(actions))), k = math.floor(len(actions)/10))
    # randomly select new actions for each position and replace
    for idx in idxs:
        actions[idx] = random.choice([0, 1, 2, 3])
    return actions

In [ ]:
def pathlen(path, target):
    # Give the first occurernce of the target in the path
    for idx, pos in enumerate(path):
        if pos == target:
            return idx+1
    return -1

In [ ]:
MAX_GENERATIONS = 1000
MAX_INDIVIDUALS = 100

best_scores =[]
best_paths = []
zero_fitness = []

print(f"> start: {start}, target: {target}")

fitness_function = lambda path: abs(path[-1][0] - target[0]) + abs(path[-1][1] - target[1])

# create a list of individuals, starting with random moves (illegal actions filtered out)
print("> Creating initial population...")
individuals = [true_random_nsteps(game_map, start, target) for _ in range(MAX_INDIVIDUALS)]
best_fitness = np.inf

print("> Evolving...")
for generation in tqdm(range(MAX_GENERATIONS)):
    
    generation_scores = []

    fitnesses = [fitness_function(individual) for individual in individuals]
    ind_actions = [actions_from_path(start, ind) for ind in individuals]
    generation_scores.append(min(fitnesses))

    # this is a list of tuples (individual, fitness). individual is a list of moves
    population = list(zip(individuals, fitnesses))
    actions =  list(zip(ind_actions, fitnesses))

    # sorting the population by best fitness (lower is better)
    population.sort(key=lambda x: x[1])
    actions.sort(key=lambda x:x[1])
    # sort ind_actions with respect to population
    
    #print(f"best score: {population[0][1]:.2f}")

    # take 2 best individuals -> maybe can be replaced with probability distribution based on fitness
    # also roulette wheel selection.

    child1, child2, = actions[0][0], actions[1][0]

    offspring = [crossover(child1, child2) for _ in range(MAX_INDIVIDUALS)]
    offspring = [mutate(child) for child in offspring]
    ind_actions = offspring
    individuals = [path_from_actions(game_map, start, child) for child in offspring]

    best_fitness = population[0][1]
    best_scores.append(population[0][1])    
    best_paths.append(population[0][0])
    #print(f"Generation {generation}: best score {best_fitness:.2f}")

    if best_fitness == 0:
        zero_fitness.append(population[0][0])
    
# print best score and best path
best_idx = np.argmin(best_scores)
print(f"Best score: {best_scores[best_idx]:.2f}")
print(f"Best path: {best_paths[best_idx]}")
print(f"generation of best path: {best_idx}")

In [ ]:
print(f'len{len(best_paths)}, path: {best_paths}')
print(f'len best path: {len(best_paths[best_idx])}, best path: {best_paths[best_idx]}')
path = best_paths[best_idx]
actions = actions_from_path(start, path)
print(f'len{len(actions)}, actions: {actions}')

In [ ]:
start

In [ ]:
for path in best_paths:
    for x, y in path:
        if is_wall(game_map[x,y]):
            raise ValueError("Path is invalid because it goes through a wall")
        


In [ ]:
env.reset()
plt.rcParams['figure.figsize'] = [17, 7]

image = plt.imshow(game[:, 300:975])
# for generation, path in enumerate(best_paths):
# plt.title(f"Generation {generation}, fitness: {best_scores[generation]:.2f}, last move: {path[-1]}")
#start = best_paths[0]
#path = best_paths[-1]
actions = []
actions = actions_from_path(start, best_paths[best_idx])

for i, action in enumerate(actions):
    try: 
        s, _, _, _ = env.step(action)
        display.display(plt.gcf())
        display.clear_output(wait=True)
        plt.title(f"Generation {generation}, fitness: {best_scores[generation]:.2f}, current position: {best_paths[best_idx][i]}, action: {action}")
        image.set_data(s['pixel'][:, 300:975])
        #time.sleep(0.1)
        if best_paths[best_idx][i] == target:
            print('YOU WON! <3')
    except RuntimeError:
        print('YOU WON! <3')

### Plot ot best fitness over generations (genetic vs random search)

In [ ]:
# We do a first comparison between the random algorithm and the genetic algorithm
MAX_GENERATIONS = 1000
MAX_INDIVIDUALS = 100

best_scores_random =[]
best_paths_random = []

print(f"> start: {start}, target: {target}")

fitness_function = lambda path: abs(path[-1][0] - target[0]) + abs(path[-1][1] - target[1])

# create a list of individuals, starting with random moves (illegal actions filtered out)
print("> Creating initial population...")
individuals_random = [true_random_nsteps(game_map, start, target) for _ in range(MAX_INDIVIDUALS)]
best_fitness_random = np.inf

print("> Evolving...")
for generation in tqdm(range(MAX_GENERATIONS)):
    
    generation_scores_random = []
    
    fitnesses_random = [fitness_function(individual) for individual in individuals_random]
    ind_actions_random = [actions_from_path(start, ind) for ind in individuals_random]
    generation_scores_random.append(min(fitnesses_random))

    # this is a list of tuples (individual, fitness). individual is a list of moves
    population_random = list(zip(individuals_random, fitnesses_random))
    actions_random =  list(zip(ind_actions_random, fitnesses_random))

    # sorting the population by best fitness (lower is better)
    population_random.sort(key=lambda x: x[1])
    actions_random.sort(key=lambda x:x[1])
    # sort ind_actions with respect to population
    

    individuals_random = [true_random_nsteps(game_map, start, target) for _ in range(MAX_INDIVIDUALS)]

    best_fitness_random = population_random[0][1]
    best_scores_random.append(population_random[0][1])    
    best_paths_random.append(population_random[0][0])
    #print(f"Generation {generation}: best score {best_fitness:.2f}")

    if best_fitness_random == 0:
        break
    
# print best score and best path
best_idx_random = np.argmin(best_scores_random)
print(f"Best score: {best_scores_random[best_idx_random]:.2f}")
print(f"Best path: {best_paths_random[best_idx_random]}")
print(f"generation of best path: {best_idx_random}")

In [ ]:
# Plot for each generation the best fitness 
plt.plot(best_scores)
# Add title and axis names
plt.title('Fitness over generations for the gentic algorithm')
plt.plot(best_scores_random)
plt.title('Fitness over generations for the random algorithm')

### Number of moves to reach the target

In [ ]:
len(best_paths[-1]), len(best_paths_random[-1]) 

In [ ]:
print(best_paths[-1])
print(best_paths_random[-1])

In [ ]:
pathlen(best_paths[-1], target)

In [ ]:
print(target)

In [ ]:
len(zero_fitness)

### 

In [ ]:
for element in zero_fitness:
    print(pathlen(element, target))

In [ ]:
min([pathlen(element, target) for element in zero_fitness])